# 01 - Dataset Manual Pre-processing

In [5]:
import math
import pandas as pd
import numpy as np
import scipy.stats as sp
from joblib import dump, load

from utils import dataframe_utils,preprocess_utils

In [6]:
df_covid = pd.read_excel('datasets/dataset.xlsx')
df_covid

,Patient ID,Patient age quantile,SARS-Cov-2 exam result,"Patient addmited to regular ward (1=yes, 0=no)","Patient addmited to semi-intensive unit (1=yes, 0=no)","Patient addmited to intensive care unit (1=yes, 0=no)",Hematocrit,Hemoglobin,Platelets,Mean platelet volume,...,Hb saturation (arterial blood gases),pCO2 (arterial blood gas analysis),Base excess (arterial blood gas analysis),pH (arterial blood gas analysis),Total CO2 (arterial blood gas analysis),HCO3 (arterial blood gas analysis),pO2 (arterial blood gas analysis),Arteiral Fio2,Phosphor,ctO2 (arterial blood gas analysis)
0,44477f75e8169d2,13,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126e9dd13932f68,17,negative,0,0,0,0.236515,-0.022340,-0.517413,0.010677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a46b4402a0e5696,8,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,f7d619a94f97c45,5,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,d9e41465789c2b5,15,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5639,ae66feb9e4dc3a0,3,positive,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5640,517c2834024f3ea,17,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5641,5c57d6037fe266d,4,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5642,c20c44766f28291,10,negative,0,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_covid.drop([
    'Patient ID',
    'Patient addmited to intensive care unit (1=yes, 0=no)',
    'Patient addmited to semi-intensive unit (1=yes, 0=no)',
    'Patient addmited to regular ward (1=yes, 0=no)'],axis = 1,inplace = True)

### Checking for missing values

In [8]:
df_counts = df_covid.count()
display(df_counts.head(56))
display(df_counts.tail(51))

Patient age quantile                                5644
SARS-Cov-2 exam result                              5644
Hematocrit                                           603
Hemoglobin                                           603
Platelets                                            602
Mean platelet volume                                 599
Red blood Cells                                      602
Lymphocytes                                          602
Mean corpuscular hemoglobin concentration (MCHC)     602
Leukocytes                                           602
Basophils                                            602
Mean corpuscular hemoglobin (MCH)                    602
Eosinophils                                          602
Mean corpuscular volume (MCV)                        602
Monocytes                                            601
Red blood cell distribution width (RDW)              602
Serum Glucose                                        208
Respiratory Syncytial Virus    

pO2 (venous blood gas analysis)              136
Fio2 (venous blood gas analysis)               1
Total CO2 (venous blood gas analysis)        136
pH (venous blood gas analysis)               136
HCO3 (venous blood gas analysis)             136
Rods #                                        97
Segmented                                     97
Promyelocytes                                 97
Metamyelocytes                                97
Myelocytes                                    97
Myeloblasts                                   97
Urine - Esterase                              60
Urine - Aspect                                70
Urine - pH                                    70
Urine - Hemoglobin                            70
Urine - Bile pigments                         70
Urine - Ketone Bodies                         57
Urine - Nitrite                                1
Urine - Density                               70
Urine - Urobilinogen                          69
Urine - Protein     

In [15]:
def drop_by_data_threshold(df,threshold,axis):
    print(f'threshold (%) = {threshold * 100}')
    if axis == 'row': axis = 1
    elif axis == 'col' or axis == 'column': axis = 0

    max_values = df.shape[axis]
    value_threshold = int(threshold * max_values)
    mask_values_to_keep = df.count(axis = axis) >= value_threshold
    print('threshold (data points) =',value_threshold)
    if axis == 1: return df[mask_values_to_keep]
    elif axis == 0: return df.loc[:,mask_values_to_keep.values]

### Drop columns with less than 10% available data

In [17]:
df_covid_subset_1 = drop_by_data_threshold(df_covid,threshold = 0.1, axis = 'column')

threshold (%) = 10.0
threshold (data points) = 564


### Drop rows with less than 90% of available features

In [20]:
df_covid_subset_2 = drop_by_data_threshold(df_covid_subset_1,threshold = 0.9, axis = 'row')
df_covid_subset_2.count()

threshold (%) = 90.0
threshold (data points) = 31


Patient age quantile                                366
SARS-Cov-2 exam result                              366
Hematocrit                                          366
Hemoglobin                                          366
Platelets                                           366
Mean platelet volume                                363
Red blood Cells                                     366
Lymphocytes                                         366
Mean corpuscular hemoglobin concentration (MCHC)    366
Leukocytes                                          366
Basophils                                           366
Mean corpuscular hemoglobin (MCH)                   366
Eosinophils                                         366
Mean corpuscular volume (MCV)                       366
Monocytes                                           365
Red blood cell distribution width (RDW)             366
Respiratory Syncytial Virus                         366
Influenza A                                     

### Drop columns with less than 80% available data

In [19]:
df_covid_subset_3 = drop_by_data_threshold(df_covid_subset_2,threshold = 0.8, axis = 'column')
df_covid_subset_3.count()

threshold (%) = 80.0
threshold (data points) = 292


Patient age quantile                                366
SARS-Cov-2 exam result                              366
Hematocrit                                          366
Hemoglobin                                          366
Platelets                                           366
Mean platelet volume                                363
Red blood Cells                                     366
Lymphocytes                                         366
Mean corpuscular hemoglobin concentration (MCHC)    366
Leukocytes                                          366
Basophils                                           366
Mean corpuscular hemoglobin (MCH)                   366
Eosinophils                                         366
Mean corpuscular volume (MCV)                       366
Monocytes                                           365
Red blood cell distribution width (RDW)             366
Respiratory Syncytial Virus                         366
Influenza A                                     

### Drop NAs

In [33]:
df_covid_clean = df_covid_subset_3.dropna()
df_covid_clean.count()

Patient age quantile                                362
SARS-Cov-2 exam result                              362
Hematocrit                                          362
Hemoglobin                                          362
Platelets                                           362
Mean platelet volume                                362
Red blood Cells                                     362
Lymphocytes                                         362
Mean corpuscular hemoglobin concentration (MCHC)    362
Leukocytes                                          362
Basophils                                           362
Mean corpuscular hemoglobin (MCH)                   362
Eosinophils                                         362
Mean corpuscular volume (MCV)                       362
Monocytes                                           362
Red blood cell distribution width (RDW)             362
Respiratory Syncytial Virus                         362
Influenza A                                     

In [34]:
df_covid_clean.shape

(362, 33)

This heuristic aimed to keep the highest number of features without losing much data points

### Visualizing categorical data values

In [64]:
def get_categorical_features(df,sort = True):
    categorical_columns = df.dtypes[df.dtypes == object].index
    if sort: return np.sort(categorical_columns)
    return categorical_columns

In [65]:
categorical_columns = get_categorical_features(df_covid_clean)
unique_values = dataframe_utils.get_column_categories(df_covid_clean[categorical_columns])

Column Values:
Adenovirus : ['not_detected' 'detected']
Bordetella pertussis : ['not_detected']
Chlamydophila pneumoniae : ['not_detected']
Coronavirus HKU1 : ['not_detected' 'detected']
Coronavirus229E : ['not_detected' 'detected']
CoronavirusNL63 : ['not_detected' 'detected']
CoronavirusOC43 : ['not_detected' 'detected']
Inf A H1N1 2009 : ['not_detected' 'detected']
Influenza A : ['not_detected' 'detected']
Influenza B : ['not_detected' 'detected']
Metapneumovirus : ['not_detected' 'detected']
Parainfluenza 1 : ['not_detected']
Parainfluenza 2 : ['not_detected']
Parainfluenza 3 : ['not_detected' 'detected']
Parainfluenza 4 : ['not_detected' 'detected']
Respiratory Syncytial Virus : ['not_detected' 'detected']
Rhinovirus/Enterovirus : ['detected' 'not_detected']
SARS-Cov-2 exam result : ['negative' 'positive']


### Dropping irrelevant columns (with only one value)

In [66]:
irrelevant_columns = []
for key in unique_values.keys():
    if unique_values[key].shape[0] <= 1:
        irrelevant_columns.append(key)
print('irrelevant_columns: ',irrelevant_columns)
df_covid_final_cut = df_covid_clean.drop(columns = irrelevant_columns)

irrelevant_columns:  ['Bordetella pertussis', 'Chlamydophila pneumoniae', 'Parainfluenza 1', 'Parainfluenza 2']


In [69]:
categorical_columns = get_categorical_features(df_covid_final_cut)
numeric_columns = np.setdiff1d(df_covid_final_cut.columns,categorical_columns)
unique_values = dataframe_utils.get_column_categories(df_covid_final_cut[categorical_columns])

Column Values:
Adenovirus : ['not_detected' 'detected']
Coronavirus HKU1 : ['not_detected' 'detected']
Coronavirus229E : ['not_detected' 'detected']
CoronavirusNL63 : ['not_detected' 'detected']
CoronavirusOC43 : ['not_detected' 'detected']
Inf A H1N1 2009 : ['not_detected' 'detected']
Influenza A : ['not_detected' 'detected']
Influenza B : ['not_detected' 'detected']
Metapneumovirus : ['not_detected' 'detected']
Parainfluenza 3 : ['not_detected' 'detected']
Parainfluenza 4 : ['not_detected' 'detected']
Respiratory Syncytial Virus : ['not_detected' 'detected']
Rhinovirus/Enterovirus : ['detected' 'not_detected']
SARS-Cov-2 exam result : ['negative' 'positive']


### Encode categorical features

In [70]:
for col in unique_values.keys():
    df_covid_final_cut.loc[df_covid_final_cut[col] == 'detected',col] = 1
    df_covid_final_cut.loc[df_covid_final_cut[col] == 'not_detected',col] = 0

df_covid_final_cut.loc[df_covid_final_cut[class_label] == 'positive',class_label] = 1
df_covid_final_cut.loc[df_covid_final_cut[class_label] == 'negative',class_label] = 0

for col in df_covid_final_cut.columns:
    if df_covid_final_cut[col].dtypes == np.float:
        continue
    df_covid_final_cut[col] = df_covid_final_cut[col].astype(int)

In [71]:
df_covid_final_cut.to_csv('datasets/dataset_preprocessed.csv',index = False)